In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "tabular_data.parquet")
df

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,target
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-29,190 Morgan,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-30,190 Morgan,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-01-31,190 Morgan,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-01,190 Morgan,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-02,190 Morgan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137083,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-27,SYS038,0
137084,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-28,SYS038,0
137085,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-29,SYS038,0
137086,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-03-30,SYS038,0


In [4]:
from datetime import datetime

from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=datetime(2024, 3, 15, 0, 0, 0),
    target_column="target"
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(100096, 674)
(100096,)
(36992, 674)
(36992,)


In [5]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]

In [6]:
import lightgbm as lgb
model = lgb.LGBMRegressor()
model.fit(X_train_only_numeric, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.343393 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 20877
[LightGBM] [Info] Number of data points in the train set: 100096, number of used features: 672
[LightGBM] [Info] Start training from score 0.382932


LGBMRegressor()

In [7]:
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_test_only_numeric)

In [8]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")

0.4955


In [9]:
from src.experiments_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv
import os
load_dotenv() 

mlflow = set_mlflow_tracking()
log_model_to_mlflow(model, X_test_only_numeric, "LGBMRegressorCitiBike", "mean_absolute_error", score=test_mae)

INFO:src.experiments_utils:MLflow tracking URI and credentials set.
2025/05/10 11:10:23 INFO mlflow.tracking.fluent: Experiment with name 'LGBMRegressorCitiBike' does not exist. Creating a new experiment.
INFO:src.experiments_utils:Experiment set to: LGBMRegressorCitiBike
INFO:src.experiments_utils:Logged mean_absolute_error: 0.4954569196274129
f:\DushyanthData\Applied_Machine_Learning\Final_Project\finalProject\lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integer

2025/05/10 11:12:30 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORK_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Registered model 'LGBMRegressor' already exists. Creating a new version of this model...
2025/05/10 11:14:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LGBMRegressor, version 2
Created version '2' of model 'LGBMRegressor'.
INFO:src.experiments_utils:Model logged with name: LGBMRegressor


🏃 View run wise-bug-295 at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/8/runs/813c194ca25c4dfd96f789aaadd7a3d4
🧪 View experiment at: https://dagshub.com/dushyanth9652narravula/dnarravu_NYC_Yellow_Cab_Taxi_Project.mlflow/#/experiments/8
